## Probabilistic Sharpe Ratio

The problem of the sharpe ratio is, that it is calculated via historical data and thus it only yields an estimation and not the true sharpe ratio!

To tackle this problem you can use the *Probabilistic Sharpe Ratio* 
It is defined as 
$$ \text{cdf} \Big( \frac{(\widehat{\text{SR}} - SR^*)}{\widehat{\sigma}({\widehat{SR})}} \Big)$$
where cdf is the cummulative density function of the normal function, <br />
$\widehat{SR}$ is the original sharpe ratio,<br />
$SR^*$ is the benchmark sharpe ratio (often 0), <br />
and $\hat{\sigma}(\widehat{SR})$ is the standard deviation of the estimated sharpe ratio which is computed as follows:

$$ \hat{\sigma}(\hat{SR}) = \sqrt{\frac{1}{n-1} \Big (1 + \frac{1}{2} \widehat{\text{SR}}^2 - \gamma_3 \hat{\text{SR}} + \frac{\gamma_4}{4} \widehat{\text{SR}}^2 \Big )} $$

Here, $\gamma_3$ and $\gamma_4$ correspond to skew and fisher kurtosis.

In words, the probabilistic sharpe ratio computes the propability that the true sharpe ratio is $\leq$ the estimated sharpe ratio ($PSR = P(SR \leq \widehat{\text{SR}})$) given the benchmark sharpe ratio.

To compute skew kurtosis and cdf, we can use scipy.stats

In [1]:
import pandas as pd

In [2]:
aapl = pd.read_csv('/Users/pratyush/Python/Python for Finance and Algorithmic Trading/Python-Finance-QuantConnect (1)/06-Financial-Concepts-with-Python/apple.csv', index_col = 'Date', parse_dates = True)

In [3]:
msft = pd.read_csv('/Users/pratyush/Python/Python for Finance and Algorithmic Trading/Python-Finance-QuantConnect (1)/06-Financial-Concepts-with-Python/msft.csv', index_col = 'Date', parse_dates = True)

In [17]:
aapl['Daily Return'] = aapl['Adj Close'].pct_change(1)

/var/folders/71/_0kmpcqd1bncrjjx5q3_sq2r0000gn/T/ipykernel_888/2882418213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl['Daily Return'] = aapl['Adj Close'].pct_change(1)


In [18]:
msft['Daily Return'] = msft['Adj Close'].pct_change(1)

In [19]:
aapl = aapl.dropna()

In [20]:
msft = msft.dropna()

In [21]:
aapl

,Open,High,Low,Close,Adj Close,Volume,Daily Return
Date,,,,,,,
2016-01-05,26.437500,26.462500,25.602501,25.677500,23.678219,223164000,-0.025059
2016-01-06,25.139999,25.592501,24.967501,25.174999,23.214844,273829600,-0.019570
2016-01-07,24.670000,25.032499,24.107500,24.112499,22.235069,324377600,-0.042205
2016-01-08,24.637501,24.777500,24.190001,24.240000,22.352642,283192000,0.005288
2016-01-11,24.742500,24.764999,24.334999,24.632500,22.714584,198957600,0.016192
...,...,...,...,...,...,...,...
2021-08-23,148.309998,150.190002,147.889999,149.710007,149.710007,60131800,0.010257
2021-08-24,149.449997,150.860001,149.149994,149.619995,149.619995,48606400,-0.000601
2021-08-25,149.809998,150.320007,147.800003,148.360001,148.360001,58991300,-0.008421


In [30]:
def sharpe_ratio(df, rf = 0):
    
    mean_return = df['Daily Return'].mean()
    
    stdev = df['Daily Return'].std()
    
    sharpe_ratio = (mean_return - rf)/stdev
    
    return sharpe_ratio

In [32]:
value = sharpe_ratio(aapl)

In [33]:
value

0.07744773478651858

In [35]:
import scipy.stats

$$ \text{cdf} \Big( \frac{(\widehat{\text{SR}} - SR^*)}{\widehat{\sigma}({\widehat{SR})}} \Big)$$


$$ \hat{\sigma}(\hat{SR}) = \sqrt{\frac{1}{n-1} \Big (1 + \frac{1}{2} \widehat{\text{SR}}^2 - \gamma_3 \hat{\text{SR}} + \frac{\gamma_4}{4} \widehat{\text{SR}}^2 \Big )} $$

In [59]:
def compute_psr(df, benchmark = 0):
    
    sr = sharpe_ratio(df)
    
    skew = scipy.stats.skew(df['Daily Return'])
    
    kurtosis = scipy.stats.kurtosis(df['Daily Return'], fisher = True)
    
    n = len(df)
    
    
    
    sigma_sr = ((1/(n-1)) * (1 + 0.5*sr**2 - skew*sr + (kurtosis/4)*sr**2)) ** 0.5
    
    final_ratio = (sr - benchmark) / sigma_sr

    psr = scipy.stats.norm.cdf(final_ratio)
    
#How do you find the norm CDF in Python?

#The easiest way to calculate normal CDF probabilities in Python is to use the norm. cdf() function 
#from the SciPy library.

#What is this? The probability that a random variables takes on a value less than 1.96 in a standard normal 
#distribution is roughly 0.975.
    
    
    return psr
    
    

In [60]:
psr_aapl = compute_psr(aapl)

In [61]:
psr_aapl

0.9980809273669636

In [62]:
#For annualized psr

psr_aapl * (252**0.5)

15.84404353277824

In [63]:
psr_msft = compute_psr(msft)

In [64]:
psr_msft * (252**0.5)

15.858321048744315